In [59]:
!pip install kagglehub

In [60]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("spscientist/students-performance-in-exams")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/students-performance-in-exams


In [61]:
import pandas as pd
import os

In [62]:
csv_file = os.path.join(path, "StudentsPerformance.csv") # Dosya birleştirme işlemi.
df = pd.read_csv(csv_file)
df.head()


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [63]:
df["average score"]=(df["math score"]+df["reading score"]+df["writing score"])/3
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,average score
0,female,group B,bachelor's degree,standard,none,72,72,74,72.666667
1,female,group C,some college,standard,completed,69,90,88,82.333333
2,female,group B,master's degree,standard,none,90,95,93,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.333333
4,male,group C,some college,standard,none,76,78,75,76.333333


In [64]:
df["passed"]=df["average score"].apply(lambda x:1 if x>=60 else 0)
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,average score,passed
0,female,group B,bachelor's degree,standard,none,72,72,74,72.666667,1
1,female,group C,some college,standard,completed,69,90,88,82.333333,1
2,female,group B,master's degree,standard,none,90,95,93,92.666667,1
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.333333,0
4,male,group C,some college,standard,none,76,78,75,76.333333,1


In [65]:
df.isnull().sum() # tüm sütunlarda kaç tane eksik değer olduğunu gösterir.
df=df[["gender","race/ethnicity","test preparation course","math score","reading score","writing score","average score","passed"]]
df["gender"]=df["gender"].map({"female":0,"male":1})
df["test preparation course"]=df["test preparation course"].map({"none":0,"completed":1})


/tmp/ipython-input-65-3928083056.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["gender"]=df["gender"].map({"female":0,"male":1})
/tmp/ipython-input-65-3928083056.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["test preparation course"]=df["test preparation course"].map({"none":0,"completed":1})


In [66]:
# Şimdi race/ethnicity sütunu için sayıya çevirme işlemi yapacağız. map ile yapılamaz.

# df["race/ethnicity"].value_counts() # Hangi gruptan kaç tane var onu gördük.

df = pd.get_dummies(df, columns=["race/ethnicity"], drop_first=True)
df.head()


,gender,test preparation course,math score,reading score,writing score,average score,passed,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E
0,0,0,72,72,74,72.666667,1,True,False,False,False
1,0,1,69,90,88,82.333333,1,False,True,False,False
2,0,0,90,95,93,92.666667,1,True,False,False,False
3,1,0,47,57,44,49.333333,0,False,False,False,False
4,1,0,76,78,75,76.333333,1,False,True,False,False


In [67]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [68]:
X=df[["gender","test preparation course","math score","reading score","writing score","race/ethnicity_group B","race/ethnicity_group C","race/ethnicity_group D","race/ethnicity_group E"]]
y=df["passed"]

In [69]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.4,random_state=42)

In [70]:
model=LogisticRegression()
model.fit(X_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [71]:
test_et=model.predict(X_test)
accuracy=accuracy_score(y_test,test_et)
print("Doğruluk oranı: ",accuracy*100)

Doğruluk oranı:  100.0


In [79]:
ornek_veri=pd.DataFrame([[1,0,55,77,60,0,0,0,1]],columns=["gender","test preparation course","math score","reading score","writing score","race/ethnicity_group B","race/ethnicity_group C","race/ethnicity_group D","race/ethnicity_group E"])
tahmin=model.predict(ornek_veri)
print("Tahmin sonucu: ", tahmin[0])
olasılık=(model.predict_proba(ornek_veri))*100
print("Olasılık: ",olasılık[0])

Tahmin sonucu:  1
Olasılık:  [ 0.3060104 99.6939896]
